In [183]:
!pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=533bf90311c680178ecf9d961274dcff33313a8855b8517b2c2e79fd8932bf7a
  Stored in directory: /Users/jeongda-eun/Library/Caches/pip/wheels/a0/b8/b7/8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent


In [3]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import json
import requests
from selenium.webdriver.common.action_chains import ActionChains



In [86]:
def scroll_down():
    pre_height = driver.execute_script("return document.body.scrollHeight") # 현재 스크롤 위치 저장
    try_num = 0
    while True :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 가장 아래로 내린다
        time.sleep(1)
        cur_height = driver.execute_script("return document.body.scrollHeight")  # 현재 스크롤을 저장한다.
        try_num += 1
        # print(try_num)
        if pre_height == cur_height :
            break
        else:
            pre_height = cur_height
    driver.implicitly_wait(100)
    
    

def go_page(region):
    # url입력
    url = "https://www.yogiyo.co.kr/mobile/#" # 사이트 입력
    driver.get(url) # 사이트 오픈
    driver.maximize_window() # 전체장
    driver.implicitly_wait(1) # 2초 지연

    # 검색창 선택
    xpath = '''//*[@id="search"]/div/form/input'''  # 검색창
    element = driver.find_element_by_xpath(xpath)
    element.clear()
    driver.implicitly_wait(1)

    # 검색창 입력

    # value = input("지역을 입력하세요")
    value = region
    element.send_keys(value)
    driver.implicitly_wait(1)

    # 검색버튼 클릭
    search_xpath = '''//*[@id="button_search_address"]/button[2]'''
    search = driver.find_element_by_xpath(search_xpath)
    driver.execute_script("arguments[0].click();", search)


    # 검색 콤보상자 선택
    search_result_selector = '#search > div > form > ul > li:nth-child(3) > a'
    search_result = driver.find_element_by_css_selector(search_result_selector)
    driver.execute_script("arguments[0].click();", search_result)
    driver.implicitly_wait(3)


def get_info():
    errors = 0
    # 페이지 소스 출력
    html = driver.page_source
    html_source = BeautifulSoup(html, 'html.parser')
    # print(html_source)
    store_list = []
    stores = driver.find_elements_by_class_name('col-sm-6')
    stores_num = len(list(set(html_source.find_all('div', class_='restaurant-name ng-binding'))))
    for s in stores:
        store_list.append(s)
        # print(s.text)
    # print(len(store_list))
    stores_num = len(list(set(stores)))
    stores_num = list(range(stores_num))
    print('stores_num : ', stores_num)
    scroll_down()
    return stores_num

def goto_store(num):
    cnt_error = 0
    try:
        scroll_down()
        #상점 이동
        in_store_xpath = f'''//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]'''
        in_store = driver.find_element_by_xpath(in_store_xpath)
        time.sleep(2)
        store_name = in_store.text
        print('store_name :', store_name)
        # in_store.click()
        driver.execute_script("arguments[0].click();", in_store)
        # ActionChains(driver).double_click(in_store)
        driver.implicitly_wait(3)
        #리뷰 탭 들어가기
        review_btn_xpath = '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a'
        review_btn = driver.find_element_by_xpath(review_btn_xpath)
        driver.execute_script("arguments[0].click();", review_btn)
        driver.implicitly_wait(3)
        print(f'{store_name} get review btn')
        #리뷰 더보기
        i = 0
        review_errors = 0
        while True:
            i += 1
            var = i*10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'
            # print(plus_btn_xpath)
            try:
                plus_btn = driver.find_element_by_xpath(plus_btn_xpath)
                # plus_btn.click()
                driver.execute_script("arguments[0].click();", plus_btn)
                print(i)
            except Exception as error:
                review_error_string = str(error)
                print(review_error_string)
                review_errors += 1
            if review_errors > 2:
                break
    except Exception as error:
        error_string = str(error)
        print(error_string)
    return store_name
        
def get_reviews(store_name):
    store_name = store_name
    html = driver.page_source
    html_source = BeautifulSoup(html, 'html.parser')
    reviews = html_source.find_all('p', attrs={'ng-show':'review.comment',
                                              'ng-bind-html':'review.comment|strip_html'})
    reviews = [r.text for r in reviews]
    taste_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_taste > 0"})
    taste_star = [t.text for t in taste_star]
    quantity_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_quantity > 0"})
    quantity_star = [q.text for q in quantity_star]
    delivery_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_delivery > 0"})
    delivery_star = [d.text for d in delivery_star]
    stars = {'taste':taste_star, 'quantity':quantity_star, 'delivery':delivery_star}
    driver.implicitly_wait(2)
      
    return reviews, stars
    # return store_list, review_list, star_list, raw_list

In [98]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5376e Safari/8536.25")
options.add_argument("--start-maximized")

# headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36",}
# region = '횡성읍'

# go_page(region)
# stores_num = get_info()
# print(stores_num)
# get_reviews(stores_num)

def before_get_review(region):
    go_page(region)
    store_num = get_info()
    return store_num


def get_total_data(num): 
    driver.implicitly_wait(2)
    before_get_review(region)
    store_name=goto_store(num)
    print(f"{num+1} go to store completed")
    reviews, stars = get_reviews(store_name)
    driver.back()
    driver.implicitly_wait(3)
    print(num+1, '뒤로가기 완료')
    print(f'finish {store_name} job completed') 
    return store_name, reviews, stars




 

In [4]:
from tqdm import tqdm
from test import get_total_data, before_get_review
from multiprocessing import Pool
import multiprocessing as multi

driver = webdriver.Chrome(executable_path='../chromedriver') 

region = '구의동'
store_num = before_get_review(region)
driver.close()

with Pool(4) as pool:
    imap = pool.map(get_total_data, [[region, i] for i in store_num])
    result = list(tqdm(imap, total=len(store_num), desc='processing'))
print(result)

stores_num :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
stores_num :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
store_name : 청춘한쌈
청춘한쌈 get review btn
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[192]/a"}
  (Session info: chrome=97.0.4692.99)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[202]/a"}
  (Session info: chrome=97.0.4692.99)

Message: no such element: Unable to locate element: {"method":"xpath","select

processing: 100%|██████████| 60/60 [00:00<00:00, 798915.05it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




In [8]:
len(list(result[0][2].values())[2])

2372

In [9]:
list(result[0][2].values())

[['5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '4',
  '4',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '3',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '5',
  '5',

In [ ]:
len(result[0][1])

In [ ]:
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame()
for i in tqdm(range(len(result))):
    name = result[i][0]
    reviews = result[i][1]
    stars = result[i][2]
    # print(stars)
    for idx in tqdm(range(len(reviews))):
        try:
            rev = reviews[idx]
            df = df.append({'name':name, 'reviews':rev}, ignore_index=True)
        except:
            pass
print(df)

100%|██████████| 8/8 [00:00<00:00, 1121.73it/s]

100%|██████████| 82/82 [00:00<00:00, 1200.34it/s]

100%|██████████| 8/8 [00:00<00:00, 1061.85it/s]

 23%|██▎       | 14/60 [00:09<00:13,  3.48it/s]
0it [00:00, ?it/s]

100%|██████████| 29/29 [00:00<00:00, 1075.51it/s]

100%|██████████| 4/4 [00:00<00:00, 891.27it/s]

100%|██████████| 96/96 [00:00<00:00, 1162.00it/s]

100%|██████████| 9/9 [00:00<00:00, 1041.29it/s]

100%|██████████| 11/11 [00:00<00:00, 828.14it/s]

 43%|████▎     | 26/60 [00:12<00:09,  3.61it/s]
0it [00:00, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 997.10it/s]

100%|██████████| 5/5 [00:00<00:00, 809.59it/s]

100%|██████████| 6/6 [00:00<00:00, 981.85it/s]

100%|██████████| 8/8 [00:00<00:00, 976.44it/s]

100%|██████████| 60/60 [01:55<00:00,  1.92s/it]

            name                                            reviews
0      춘리마라탕-구의점  맛있었습니다ㅜㅜㅜ알바끝나고 먹는 마라탕짱.. 저 많은 가게중에 여기서만 ㅅㅣ키는데 ...
1      춘리마라탕-구의점                          서비스 감사합니당~ 맛있게 잘 먹었어요👍🏻👍🏻
2      춘리마라탕-구의점              연휴에 배달해주셔서 감사합니다 배달도 예정시간보다 빠르고 맛있어요~
3      춘리마라탕-구의점                           매운게 땡겨서 주문했어요~ 잘 먹었습니다!!
4      춘리마라탕-구의점                                 배달이 생각보다 빨리와서 좋았네요
...          ...                                                ...
86315   찜닭&닭도리명가  진짜 맛있어요 간만에 맛있는 찜닭집 찾아서 넘 행복 ㅠㅠ 좀 기름기 있는편이지만 맛...
86316   찜닭&닭도리명가                                    너무 달지도 맵지도 딱좋아요
86317   찜닭&닭도리명가                                         맛있었어요 굳굳ㅎㅎ
86318   찜닭&닭도리명가                             배달도 빠르고 맛도 있어요 \n대박나세요
86319   찜닭&닭도리명가     정말 맛있네요\n목이버섯 별로 안좋아하는데\n국물이 너무 맛있어서 다먹었어요\n굿굿

[86320 rows x 2 columns]


In [16]:
df['reviews'][10]

'꿔바로우는 고기가 너무 얇고 튀김옷만 두꺼움 솔직히 돈 아깝다는 생각이 먹는 내내 들었습니다 \n\n다른 음식은 제가 처음 먹는거라 비교불가하니 노코맨트 하겠습니다'

In [17]:
def cleanhtml(raw_html):
    cleanr = re.compile('\n')
    # cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [18]:
df['reviews'] = df['reviews'].apply(lambda x:cleanhtml(x))

In [19]:
df['reviews'][10]

'꿔바로우는 고기가 너무 얇고 튀김옷만 두꺼움 솔직히 돈 아깝다는 생각이 먹는 내내 들었습니다 다른 음식은 제가 처음 먹는거라 비교불가하니 노코맨트 하겠습니다'

In [20]:
df.to_csv('./reviews.csv', encoding='utf-8=sig')